# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

In [8]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [9]:
epochs = 500  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [10]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization_1 (Normalization  (None, 8)           17          ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 dense_2 (Dense)                (None, 16)           144         ['normalization_1[0][0]']        
                                                                                                  
 dense_3 (Dense)                (None, 16)           272         ['dense_2[0][0]']          

### Fitting

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/500


3122/3122 - 6s - loss: 54594.9922 - output1_loss: 27058.8203 - output_loss: 27536.2305 - val_loss: 42545.1406 - val_output1_loss: 21275.0820 - val_output_loss: 21270.0781 - 6s/epoch - 2ms/step
Epoch 2/500
3122/3122 - 6s - loss: 42018.0469 - output1_loss: 21010.1250 - output_loss: 21007.9258 - val_loss: 41726.2188 - val_output1_loss: 20862.5059 - val_output_loss: 20863.7441 - 6s/epoch - 2ms/step
Epoch 3/500
3122/3122 - 7s - loss: 41640.4531 - output1_loss: 20819.8594 - output_loss: 20820.6113 - val_loss: 41585.7891 - val_output1_loss: 20792.6875 - val_output_loss: 20793.0996 - 7s/epoch - 2ms/step
Epoch 4/500
3122/3122 - 6s - loss: 41580.5938 - output1_loss: 20790.2148 - output_loss: 20790.3750 - val_loss: 41569.2031 - val_output1_loss: 20784.6172 - val_output_loss: 20784.6094 - 6s/epoch - 2ms/step
Epoch 5/500
3122/3122 - 5s - loss: 41573.7852 - output1_loss: 20786.9102 - output_loss: 20786.8945 - val_loss: 41566.4961 - val_output1_loss: 20783.2793 - val_output_loss: 20783.2363 - 5s/epoc

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [12]:
fit = np.load("data/fit.npz")

In [13]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)